# 🛡️ NIDS-ML Pipeline - Kaggle (Strict & Optimized)

Versione finale dell'architettura per il training su Kaggle.

### Caratteristiche:
- **Clean Run:** Configurabile per pulire o mantenere i risultati precedenti.
- **Dataset Copy:** Importazione stabile dei dati (no symlink).
- **Strict Mode:** Chiamate agli script senza parametri superflui.
- **Repo Management:** Download e setup pulito della sola cartella `src/`.

In [1]:
# ==========================================
# 1. CONFIGURAZIONE (STRICT)
# ==========================================

# --- 1.1 Configurazione Pipeline ---
CLEAN_RUN = True      # Se True: Cancella data/processed, artifacts e tuning_results prima di iniziare

# --- 1.2 Configurazione Tuning ---
MODEL_TYPE = "lightgbm"    # Opzioni: random_forest,xgboost,lightgbm
N_TRIALS = 60    # Numero di trial per Optuna
TIMEOUT = 60       # Timeout in secondi (es. 3600 = 1 ora)

# --- 1.3 Configurazione Ambiente ---
REPO_URL = "https://github.com/riiccardob/nids-ml-ssr2"
BRANCH = "main"
KAGGLE_INPUT_PATH = "/kaggle/input/network-intrusion-dataset/" # Path del dataset su Kaggle

# ==========================================
# VALIDAZIONE PARAMETRI
# ==========================================
ALLOWED_MODELS = {"random_forest", "xgboost", "lightgbm"}
if MODEL_TYPE not in ALLOWED_MODELS:
    raise ValueError(f"❌ Modello '{MODEL_TYPE}' non valido. Scegli tra: {ALLOWED_MODELS}")

if N_TRIALS is None and TIMEOUT is None:
    raise ValueError("❌ Devi specificare almeno uno tra N_TRIALS o TIMEOUT.")

print(f"✅ Configurazione valida: Modello={MODEL_TYPE}, CleanRun={CLEAN_RUN}")

✅ Configurazione valida: Modello=lightgbm, CleanRun=True


In [2]:
# ==========================================
# 2. SETUP AMBIENTE (Repo & Requirements)
# ==========================================
import os
import shutil
import sys

# Rileva ambiente Kaggle
KAGGLE_ENV = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '') != ''

if KAGGLE_ENV:
    print("🚀 Ambiente Kaggle rilevato. Setup repo...")
    
    repo_folder = f"NIDS-ML-SSR2-{BRANCH}"
    zip_name = "repo.zip"
    
    # Scarica solo se src non esiste o se vogliamo forzare un aggiornamento pulendo manualmente
    # Nota: Qui src viene mantenuto tra le run se non esplicitamente cancellato, 
    # ma per sicurezza riscarichiamo se manca.
    if not os.path.exists("src"):
        print(f"📥 Download repo da {REPO_URL}...")
        os.system(f"wget -q {REPO_URL}/archive/refs/heads/{BRANCH}.zip -O {zip_name}")
        os.system(f"unzip -qo {zip_name}")
        
        # Setup cartelle
        if os.path.exists(f"{repo_folder}/src"):
            shutil.move(f"{repo_folder}/src", "./src")
        if os.path.exists(f"{repo_folder}/requirements.txt"):
            shutil.move(f"{repo_folder}/requirements.txt", "./requirements.txt")
            
        # Cleanup download
        if os.path.exists(repo_folder): shutil.rmtree(repo_folder)
        if os.path.exists(zip_name): os.remove(zip_name)
    
    # Installazione dipendenze
    if os.path.exists("requirements.txt"):
        print("🔧 Installazione dipendenze...")
        os.system("pip install -q -r requirements.txt")
        # Fix specifico per Kaggle (conflitto dill/datasets)
        os.system("pip install -q --upgrade datasets")
        
    sys.path.append(os.path.abspath("src"))
    print("✅ Setup ambiente completato.")
else:
    print("💻 Ambiente Locale. Uso i file esistenti.")

🚀 Ambiente Kaggle rilevato. Setup repo...
📥 Download repo da https://github.com/riiccardob/nids-ml-ssr2...
🔧 Installazione dipendenze...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 5.2 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.4.2 requires dill<0.4.1,>=0.3.0, but you have dill 0.4.1 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 5.3 MB/s eta 0:00:00
✅ Setup ambiente completato.


In [3]:
# ==========================================
# 3. CLEAN RUN MANAGER
# ==========================================
# Gestisce la pulizia delle cartelle di output in base a CLEAN_RUN

DIRS_TO_CLEAN = ["data/processed", "artifacts", "tuning_results"]

if CLEAN_RUN:
    print("🧹 CLEAN_RUN = True: Pulizia cartelle di output...")
    for directory in DIRS_TO_CLEAN:
        if os.path.exists(directory):
            print(f'  - Rimozione: {directory}')
            shutil.rmtree(directory)
    print("✅ Pulizia completata.")
else:
    print("⏩ CLEAN_RUN = False: I risultati precedenti vengono mantenuti.")

🧹 CLEAN_RUN = True: Pulizia cartelle di output...
✅ Pulizia completata.


In [4]:
# ==========================================
# 4. IMPORT DATASET (COPY STRATEGY)
# ==========================================

RAW_DIR = "data/raw"
os.makedirs(RAW_DIR, exist_ok=True)

print(f"📦 Controllo Dataset in {RAW_DIR}...")

if KAGGLE_ENV:
    if not os.path.exists(KAGGLE_INPUT_PATH):
        print(f"⚠️ ERRORE: Path {KAGGLE_INPUT_PATH} non trovato!")
    else:
        copied_count = 0
        # Cerca CSV ricorsivamente
        for root, dirs, files in os.walk(KAGGLE_INPUT_PATH):
            for file in files:
                if file.lower().endswith(".csv"):
                    src_path = os.path.join(root, file)
                    dst_path = os.path.join(RAW_DIR, file)
                    
                    # Copia solo se non esiste
                    if not os.path.exists(dst_path):
                        print(f'  - Copia in corso: {file}...')
                        shutil.copy2(src_path, dst_path)
                        copied_count += 1
        
        if copied_count > 0:
            print(f"✅ Copiati {copied_count} file nuovi.")
        else:
            print("✅ Tutti i file sono già presenti in data/raw.")
            
    # DEBUG: Stampa contenuto data/raw
    print(f"\n📂 Contenuto attuale di {RAW_DIR}:")
    print(os.listdir(RAW_DIR))
else:
    print("💻 Ambiente Locale: Verifica manuale richiesta per data/raw.")

📦 Controllo Dataset in data/raw...
  - Copia in corso: Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv...
  - Copia in corso: Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv...
  - Copia in corso: Tuesday-WorkingHours.pcap_ISCX.csv...
  - Copia in corso: Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv...
  - Copia in corso: Monday-WorkingHours.pcap_ISCX.csv...
  - Copia in corso: Friday-WorkingHours-Morning.pcap_ISCX.csv...
  - Copia in corso: Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv...
  - Copia in corso: Wednesday-workingHours.pcap_ISCX.csv...
✅ Copiati 8 file nuovi.

📂 Contenuto attuale di data/raw:
['Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv', 'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', 'Tuesday-WorkingHours.pcap_ISCX.csv', 'Monday-WorkingHours.pcap_ISCX.csv', 'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv', 'Friday-WorkingHours-Morning.pcap_ISCX.csv', 'Wednesday-workingHours.pcap_ISCX.csv', 'Friday-WorkingHours-Aft

In [5]:
# ==========================================
# 5. PREPROCESSING (STRICT)
# ==========================================
# Nessun parametro passato, usa i default definiti in src/utils.py

print("⚙️ Avvio Preprocessing...")
!python src/preprocessing.py --n-jobs 4

⚙️ Avvio Preprocessing...

PREPROCESSING CIC-IDS2017

Parametri:
  Input:         /kaggle/working/data/raw
  Output:        /kaggle/working/data/processed
  Balance:       Si (ratio 2.0:1)
  Chunk size:    Disabilitato
  Split:         70/15/15
  CPU cores:     4/4

1. Caricamento CSV da /kaggle/working/data/raw...
2026-01-25 19:42:55 | INFO     | Trovati 8 file CSV
Caricamento CSV: 100%|████████████████████████████| 8/8 [00:39<00:00,  4.97s/it]
2026-01-25 19:43:35 | INFO     | Concatenazione 8 DataFrame...
2026-01-25 19:43:36 | INFO     | Dataset combinato: 2,830,743 righe, 79 colonne
2026-01-25 19:43:37 | INFO     | Memoria: 1.8 GB

2. Pulizia dati...
2026-01-25 19:43:37 | INFO     | Inizio pulizia dati...
2026-01-25 19:43:37 | INFO     | Rimosse 1 colonne identificative
2026-01-25 19:43:44 | INFO     | Rimosse 2,867 righe con valori infiniti
2026-01-25 19:44:17 | INFO     | Rimosse 594,712 righe duplicate
2026-01-25 19:44:18 | INFO     | Pulizia completata: 2,830,743 -> 2,233,164 ri

In [6]:
# ==========================================
# 6. FEATURE ENGINEERING (STRICT)
# ==========================================
# Nessun parametro passato

print("🧠 Avvio Feature Engineering...")
!python src/feature_engineering.py --n-jobs 4

🧠 Avvio Feature Engineering...

FEATURE ENGINEERING

Parametri:
  Feature da selezionare: 30
  RF estimators:          100
  CPU cores:              4/4
  Max RAM:                85%

1. Caricamento dati preprocessati...
2026-01-25 19:44:35 | INFO     | Caricati: train=706,632, val=151,422, test=151,422
   Train: 706,632 | Val: 151,422 | Test: 151,422
2026-01-25 19:44:35 | INFO     | CPU: 4.9% | RAM: 7.7% | Disponibile: 28.9GB | Core attivi: 4/4

2. Esecuzione pipeline feature engineering...
2026-01-25 19:44:35 | INFO     | Feature iniziali: 77
2026-01-25 19:44:35 | INFO     | Fitting scaler su 706,632 campioni, 77 feature
2026-01-25 19:44:37 | INFO     | Training RF per selezione feature (n_estimators=100, n_jobs=4)...
   Training RF: 100 alberi su 706,632 campioni...
   Core CPU in uso: 4
   Questo puo richiedere 1-2 minuti...
   RF training completato
2026-01-25 19:46:55 | INFO     | Selezionate 30 feature su 77
2026-01-25 19:46:56 | INFO     | Salvato: scaler.pkl
2026-01-25 19:46:5

In [7]:
###### ==========================================
# 7. HYPERPARAMETER TUNING (STRICT)
# ==========================================

cmd = [
    'python', 'src/hyperparameter_tuning.py',
    '--model', MODEL_TYPE
]

if N_TRIALS is not None:
    cmd += ['--n-trials', str(N_TRIALS)]

if TIMEOUT is not None:
    cmd += ['--timeout', str(TIMEOUT)]

cmd_str = " ".join(cmd)
print(f"🚀 Avvio Tuning: {cmd_str}")

!{cmd_str}

🚀 Avvio Tuning: python src/hyperparameter_tuning.py --model lightgbm --n-trials 60 --timeout 60

HYPERPARAMETER TUNING

Modello:      lightgbm
Metodo:       Bayesian Optimization (Optuna)
Metrica:      70% F2-Score + 30% Latency (composite)
Task:         binary
CV:           5
Max Latency:  1.0ms/sample
CPU:          2/4
Limiti:       60 trials OPPURE 60s (0.0h)
              (si ferma al primo raggiunto)

1. Caricamento dati...
2026-01-25 19:47:09 | INFO     | Caricati: train=706,632, val=151,422, test=151,422
2. Preparazione feature...
2026-01-25 19:47:09 | INFO     | Caricati artifacts da /kaggle/working/artifacts
   Shape: (706632, 30)

3. Tuning (Bayesian Optuna)...
   NOTA: Misura latency durante CV, rallenta il processo
2026-01-25 19:47:10 | INFO     | Bayesian Optimization (Optuna): cv=5
2026-01-25 19:47:10 | INFO     | Metrica: 70% F2-Score + 30% Latency
2026-01-25 19:47:10 | INFO     | Max latency constraint: 1.0ms/sample
2026-01-25 19:47:10 | INFO     | Limiti: 60 trials OPP

In [8]:
# ==========================================
# 8. SALVATAGGIO OUTPUT
# ==========================================

OUTPUT_ZIP = "tuning_results_pack.zip"
DIRS_TO_SAVE = ["tuning_results", "artifacts"] # Aggiungi 'models' se necessario

print("📦 Creazione archivio finale...")

existing_dirs = [d for d in DIRS_TO_SAVE if os.path.exists(d)]

if existing_dirs:
    dirs_str = " ".join(existing_dirs)
    !zip -qr {OUTPUT_ZIP} {dirs_str}
    print(f"✅ Archivio pronto: {OUTPUT_ZIP} (Scarica File)")
    
    # Lista contenuto zip per verifica
    # !unzip -l {OUTPUT_ZIP}
else:
    print("⚠️ Nessun risultato trovato da salvare.")

📦 Creazione archivio finale...
✅ Archivio pronto: tuning_results_pack.zip (Scarica File)
